In [4]:
# Importation des modules nécessaires
import streamlit as st
import pandas as pd
import function
from datetime import datetime, timedelta,time, timezone
import numpy as np
import plotly.express as px
import requests
from io import StringIO



url = "https://kf.kobotoolbox.org/api/v2/assets/azR3wHzm3BK4YKbrrrSfFo/export-settings/esw25ALnhMxzoshajCy4hGm/data.csv"

username = "amanyankey2"
password = "L@ScXi7TFhKWrN5"

# Effectuer une requête HTTP avec authentification basique
response = requests.get(url, auth=(username, password))

# Vérifier que la requête a réussi
if response.status_code == 200:
    df = function.get_data_from_forms(StringIO(response.text))
else:
    st.warning("Aucune data")

In [18]:
df[["UEF_partiel_agent1","UEI_partiel_agent1","UEP_sans_statut_agent1","UEF_partiel_agent2","UEI_partiel_agent2","UEP_sans_statut_agent2","UEF_partiel_agent3","UEI_partiel_agent3","UEP_sans_statut_agent3"]].sum(axis=1).sum()-df[['UEPF_trans_complet_agent1','UEPI_trans_complet_agent1','UEPF_trans_complet_agent2','UEPI_trans_complet_agent2','UEPF_trans_complet_agent3','UEPI_trans_complet_agent3']].sum(axis=1).sum()

523

In [17]:
df[['UEPF_trans_complet_agent1','UEPI_trans_complet_agent1','UEPF_trans_complet_agent2','UEPI_trans_complet_agent2','UEPF_trans_complet_agent3','UEPI_trans_complet_agent3']].sum(axis=1).sum()

56

In [ ]:

#STATISTIQUE EQUIPE
st.markdown("<h5 style='text-align: center;color: #3a416c;'>STATISTIQUES PAR EQUIPES</h5>", unsafe_allow_html=True)
stat_ce = df[["Chef d'equipe","UE F","UE I","UE_total","partiels","refus","NbZD","NumZD","ZD_affectees"]]
stat_ce = stat_ce.groupby("Chef d'equipe").agg({
    "UE F": "sum",
    "UE I": "sum",
    "UE_total": "sum",
    "partiels": "sum",
    "refus": "sum",
    "NbZD": "sum",
    "NumZD": function.concat_list_zd,
    "ZD_affectees": "first"
}).reset_index()

stat_ce["NumZD"] = stat_ce["NumZD"].apply(function.delete_doublon_and_sort_from_list_zd)
stat_ce["NbZD"] = stat_ce["NumZD"].apply(function.count_unique_zd)
stat_ce["ZD_affectees"] =stat_ce["ZD_affectees"].astype("int")
stat_ce["ZD restantes"] =stat_ce["ZD_affectees"] - stat_ce["NbZD"]
stat_ce.drop(columns="NumZD",inplace=True)
stat_ce.sort_values(by="UE_total",ascending=False,inplace=True)
stat_ce.set_index("Chef d'equipe",inplace=True)
st.table(function.style_dataframe(stat_ce))


#classement des AGENTS
st.markdown("<h5 style='text-align: center;color: #3a416c;'>RESULTATS AGENTS RECENSEURS</h5>", unsafe_allow_html=True)
stat_agent_lastday = df[["date_reporting","Chef d'equipe","nom_CE","UE_agent1","UE_agent2","UE_agent3"]]
stat_agent_lastday = stat_agent_lastday.melt(id_vars=['date_reporting', 'Chef d\'equipe', "nom_CE"], 
                                             value_vars=['UE_agent1', 'UE_agent2', 'UE_agent3'], 
                                             var_name='Nom_Agent', 
                                             value_name='Value')

stat_agent_lastday = stat_agent_lastday.pivot_table(index=['Chef d\'equipe', "nom_CE", 'Nom_Agent'], 
                                                    columns='date_reporting', 
                                                    values='Value').reset_index()
numeric_columns = stat_agent_lastday.select_dtypes(include=[np.number]).columns.tolist()
# Remplir les NaN avec 0
stat_agent_lastday[numeric_columns] = stat_agent_lastday[numeric_columns].fillna(0)

# Calculer la somme pour la colonne "Total depuis le debut"
stat_agent_lastday["Total depuis le debut"] = stat_agent_lastday[numeric_columns].sum(axis=1)# Remplacer les NaN par une valeur par défaut (par exemple 0)
stat_agent_lastday[filtered_columns.strftime('%Y-%m-%d').tolist()] = stat_agent_lastday[filtered_columns.strftime('%Y-%m-%d').tolist()].fillna(0).astype(int)

stat_agent_lastday['Nom_Agent'] = stat_agent_lastday['nom_CE'].str.replace('Ce', 'Agt') + stat_agent_lastday['Nom_Agent'].str[-1]
stat_agent_lastday = function.add_agent_name(stat_agent_lastday)
stat_agent_lastday = stat_agent_lastday[['Chef d\'equipe',"nom_CE", 'Nom_Agent']+filtered_columns.strftime('%Y-%m-%d').tolist()+['Total depuis le debut']]
stat_agent_lastday['Total depuis le debut'] = stat_agent_lastday['Total depuis le debut'].astype(int)
stat_agent_lastday.drop(columns=["nom_CE"], inplace=True)
stat_agent_lastday.sort_values(by=['Chef d\'equipe', 'Nom_Agent'], ascending=True, inplace=True)



stat_agent_lastday = stat_agent_lastday.reset_index(drop=True)
stat_agent_lastday.index = stat_agent_lastday.index + 1
st.table(function.style_dataframe(stat_agent_lastday))


Index(['debut', 'fin', 'text', 'titre', 'nom_CE', 'date_reporting', 'NbZD',
       'NumZD', 'note1', 'UEF_complet_agent1', 'UEF_partiel_agent1',
       'UEI_complet_agent1', 'UEI_partiel_agent1', 'UEP_sans_statut_agent1',
       'UE_refus_agent1', 'UEPF_trans_complet_agent1',
       'UEPI_trans_complet_agent1', 'UE_total_agent1', 'note2',
       'UEF_complet_agent2', 'UEF_partiel_agent2', 'UEI_complet_agent2',
       'UEI_partiel_agent2', 'UEP_sans_statut_agent2', 'UE_refus_agent2',
       'UEPF_trans_complet_agent2', 'UEPI_trans_complet_agent2',
       'UE_total_agent2', 'note3', 'UEF_complet_agent3', 'UEF_partiel_agent3',
       'UEI_complet_agent3', 'UEI_partiel_agent3', 'UEP_sans_statut_agent3',
       'UE_refus_agent3', 'UEPF_trans_complet_agent3',
       'UEPI_trans_complet_agent3', 'UE_total_agent3', 'note5', 'UE_total',
       'note4', 'Nb_rdv', 'Nb_rdv_h', 'difficultes', 'observations', '_id',
       '_uuid', '_submission_time', '_validation_status', '_notes', '_status',
     

In [38]:
df[['UEF_complet_agent1','UEI_complet_agent1',"UEF_partiel_agent1","UEI_partiel_agent1","UEP_sans_statut_agent1","UE_refus_agent1",'UEF_complet_agent1','UEI_complet_agent2',"UEF_partiel_agent2","UEI_partiel_agent2","UEP_sans_statut_agent2","UE_refus_agent2",'UEF_complet_agent3','UEI_complet_agent3',"UEF_partiel_agent3","UEI_partiel_agent3","UEP_sans_statut_agent3","UE_refus_agent3"]].sum(axis=1).sum()

1719

In [ ]:

#Chargement du fichier d'attribution des ZD par équipes et merge avec les geodf
#df_attribution_zd = function.get_data_attribution_eq_xlsx("Attribution_zd.xlsx")
#chargement des ZDs
#geo_df = function.load_geozd("Contour_NAWA_GBOKLE.geojson")
#geo_df = geo_df.merge(df_attribution_zd[['NumZD', "NOM CHEF D'EQUIPE"]], on='NumZD', how='left')
#st.write(geo_df[["NomSp",'NumZD', "NOM CHEF D'EQUIPE"]])

#Coordonnées initiales pour l'affichage de la carte (Soubre)
#latitude_centre = 5.788289
#longitude_centre = -6.594167
Nb_ZD_initial = 704

with st.sidebar:
    if st.button("ACTUALISER", type="primary"):
        function.get_data_from_forms.clear()

    CE =df["Chef d'equipe"].sort_values().unique()
    CE_SELECT = st.selectbox("CHEFS D'EQUIPE:",CE,index=None)


if len(df)!= 0 :
    try:
        if CE_SELECT:
            df = df.loc[df["Chef d'equipe"]==CE_SELECT]
            #geo_df = geo_df.loc[geo_df["NOM CHEF D'EQUIPE"]==CE_SELECT]
            Nb_ZD_initial = function.zd_region[REG_SELECT]
    except:
        pass
    
def split_and_collect(column):
    result = []
    for item in column:
        if isinstance(item, str):  # Vérifie si l'élément est une chaîne de caractères
            result.extend(item.split(','))
        elif pd.notna(item):  # Vérifie si l'élément n'est pas NaN (valeurs manquantes)
            result.append(str(item))  # Convertit les autres types en chaîne de caractères et les ajoute à la liste
    return result


#recupeeration coordonnees cnetroide en fonction de la zone geographique


# Appliquer la fonction à la colonne
liste_zd = split_and_collect(df['NumZD'])
liste_zd = list(set(liste_zd))
liste_zd = [s for s in liste_zd if "0000" not in s]

#Identification des ZDs traiteees
#geo_df["Statut ZD"] = geo_df["NumZD"].apply(lambda x: function.statut_zd(x, liste_zd))

# Obtenez l'heure actuelle en GMT+0
current_time = datetime.now(timezone.utc).time()
current_date = pd.to_datetime(datetime.now().date())
cutoff_time = time(17, 30)  # 17:30 in 24-hour format

if current_time < cutoff_time:
    df.loc[df['date_2']!=current_date-timedelta(days=1), 'partiels'] = 0
    df.loc[df['date_2']!= current_date-timedelta(days=1), 'refus'] = 0
    
else:
    df.loc[df['date_2'] < current_date, 'partiels'] = 0
    df.loc[df['date_2'] < current_date, 'refus'] = 0

df['partiels'] = df['partiels'].astype(int)
df['refus'] = df['refus'].astype(int)


UET = df["UE_total"].sum()
UEI = df["UE I"].sum()
UEF = df["UE F"].sum()
REFUS = df["refus"].sum()
UE_partiels = df["partiels"].sum()
Nb_rdv_attente = df["Nb_rdv_attente"].sum()
Nb_rdv_h = df["Nb_rdv_h"].sum()
Nb_rdv = df["Nb_rdv"].sum()

ZD_total = len(liste_zd)
container =st.container()
with container:
    col1, col2, col3, col3a = st.columns(4)
    col1.metric("UE", f"{UET:,}")
    col2.metric("Formelle", f"{UEF:,}")
    col3.metric("Informelle", f"{UEI:,}")
    col3a.metric("Partiels", f"{UE_partiels:,}")
with container:
    col4, col5, col6 = st.columns([2,3,2])
    col4.metric("ZDs traités", f"{ZD_total:,}")
    col5.metric("Taux de réalisation ZD", f"{(ZD_total/Nb_ZD_initial)*100:.2f}%")
    col6.metric("Refus", f"{REFUS:,}")
with container:
    col1, col2, col3 = st.columns(3)
    col1.metric("Total RDV", f"{Nb_rdv:,}")
    col2.metric("RDV honoré", f"{Nb_rdv_h:,}")
    col3.metric("RDV en attente", f"{Nb_rdv_attente:,}")

#RESULTAT PAR EQUIPE SUR LES 5 DERNIERS JOURS
st.markdown("<h5 style='text-align: center;color: #3a416c;'>RESULTAT PAR EQUIPE SUR LES 5 DERNIERS JOURS</h5>", unsafe_allow_html=True)
pivot_df =  df[["date_reporting","Chef d'equipe","UE_total"]]
pivot_df = pivot_df.pivot_table(index="Chef d'equipe", columns='date_reporting',values='UE_total', aggfunc='sum', fill_value=0)
pivot_df["Total depuis le debut"] = pivot_df.sum(axis=1)

sum_row = pivot_df.sum(axis=0)
sum_row_df = pd.DataFrame(sum_row).T
sum_row_df.index = ['Total']
pivot_df = pd.concat([pivot_df, sum_row_df])

ensemble_col = pivot_df['Total depuis le debut']
date_columns = pd.to_datetime(pivot_df.columns.drop('Total depuis le debut'))

today = datetime.now()
five_days_ago = today - timedelta(days=5)
filtered_columns = date_columns[date_columns >= five_days_ago]
pivot_df = pivot_df[filtered_columns.strftime('%Y-%m-%d').tolist()+['Total depuis le debut']]
st.table(function.style_dataframe(pivot_df))

#STATISTIQUE EQUIPE
st.markdown("<h5 style='text-align: center;color: #3a416c;'>STATISTIQUES PAR EQUIPES</h5>", unsafe_allow_html=True)
stat_ce = df[["Chef d'equipe","UE F","UE I","UE_total","partiels","refus","NbZD","NumZD","ZD_affectees"]]
stat_ce = stat_ce.groupby("Chef d'equipe").agg({
    "UE F": "sum",
    "UE I": "sum",
    "UE_total": "sum",
    "partiels": "sum",
    "refus": "sum",
    "NbZD": "sum",
    "NumZD": function.concat_list_zd,
    "ZD_affectees": "first"
}).reset_index()

stat_ce["NumZD"] = stat_ce["NumZD"].apply(function.delete_doublon_and_sort_from_list_zd)
stat_ce["NbZD"] = stat_ce["NumZD"].apply(function.count_unique_zd)
stat_ce["ZD_affectees"] =stat_ce["ZD_affectees"].astype("int")
stat_ce["ZD restantes"] =stat_ce["ZD_affectees"] - stat_ce["NbZD"]
stat_ce.drop(columns="NumZD",inplace=True)
stat_ce.sort_values(by="UE_total",ascending=False,inplace=True)
stat_ce.set_index("Chef d'equipe",inplace=True)
st.table(function.style_dataframe(stat_ce))


#classement des AGENTS
st.markdown("<h5 style='text-align: center;color: #3a416c;'>RESULTATS AGENTS RECENSEURS</h5>", unsafe_allow_html=True)
stat_agent_lastday = df[["date_reporting","Chef d'equipe","nom_CE","UE_agent1","UE_agent2","UE_agent3"]]
stat_agent_lastday = stat_agent_lastday.melt(id_vars=['date_reporting', 'Chef d\'equipe', "nom_CE"], 
                                             value_vars=['UE_agent1', 'UE_agent2', 'UE_agent3'], 
                                             var_name='Nom_Agent', 
                                             value_name='Value')

stat_agent_lastday = stat_agent_lastday.pivot_table(index=['Chef d\'equipe', "nom_CE", 'Nom_Agent'], 
                                                    columns='date_reporting', 
                                                    values='Value').reset_index()
numeric_columns = stat_agent_lastday.select_dtypes(include=[np.number]).columns.tolist()
# Remplir les NaN avec 0
stat_agent_lastday[numeric_columns] = stat_agent_lastday[numeric_columns].fillna(0)

# Calculer la somme pour la colonne "Total depuis le debut"
stat_agent_lastday["Total depuis le debut"] = stat_agent_lastday[numeric_columns].sum(axis=1)# Remplacer les NaN par une valeur par défaut (par exemple 0)
stat_agent_lastday[filtered_columns.strftime('%Y-%m-%d').tolist()] = stat_agent_lastday[filtered_columns.strftime('%Y-%m-%d').tolist()].fillna(0).astype(int)

stat_agent_lastday['Nom_Agent'] = stat_agent_lastday['nom_CE'].str.replace('Ce', 'Agt') + stat_agent_lastday['Nom_Agent'].str[-1]
stat_agent_lastday = function.add_agent_name(stat_agent_lastday)
stat_agent_lastday = stat_agent_lastday[['Chef d\'equipe',"nom_CE", 'Nom_Agent']+filtered_columns.strftime('%Y-%m-%d').tolist()+['Total depuis le debut']]
stat_agent_lastday['Total depuis le debut'] = stat_agent_lastday['Total depuis le debut'].astype(int)
stat_agent_lastday.drop(columns=["nom_CE"], inplace=True)
stat_agent_lastday.sort_values(by=['Chef d\'equipe', 'Nom_Agent'], ascending=True, inplace=True)



stat_agent_lastday = stat_agent_lastday.reset_index(drop=True)
stat_agent_lastday.index = stat_agent_lastday.index + 1
st.table(function.style_dataframe(stat_agent_lastday))

#liste des ZD deja acheves
st.markdown("<h5 style='text-align: center;color: #3a416c;'>LISTES DES ZD TRAITEES</h5>", unsafe_allow_html=True)
df_zd = df[["Chef d'equipe","NumZD"]]
df_zd = df_zd.groupby("Chef d'equipe").agg({"NumZD": function.concat_list_zd}).reset_index()
df_zd["NumZD"] = df_zd["NumZD"].apply(function.delete_doublon_and_sort_from_list_zd)
df_zd.set_index("Chef d'equipe",inplace=True)

st.table(function.style_dataframe(df_zd))

#DIFFICULTES DU JOUR
yesterday = datetime.now().date()-timedelta(days=1)
st.markdown("<h5 style='text-align: center;color: #3a416c;'>DIFFICULTES DU JOUR</h5>", unsafe_allow_html=True)
df_difficultes_obs = df[["Chef d'equipe","difficultes","date_reporting","observations"]]
df_difficultes_obs["date_reporting"] = df_difficultes_obs["date_reporting"].apply(function.convert_to_datetime)
df_difficultes_obs = df_difficultes_obs[df_difficultes_obs["date_reporting"] >=yesterday]

df_difficultes = df_difficultes_obs[~df_difficultes_obs['difficultes'].isna()]
df_difficultes = df_difficultes.groupby("Chef d'equipe")['difficultes'].agg('/'.join).reset_index()
st.table(function.style_dataframe(df_difficultes))

st.markdown("<h5 style='text-align: center;color: #3a416c;'>OBSERVATIONS DU JOUR</h5>", unsafe_allow_html=True)
df_obs = df_difficultes_obs[~df_difficultes_obs['observations'].isna()]
df_obs = df_obs.groupby("Chef d'equipe")['observations'].agg('/'.join).reset_index()
st.table(function.style_dataframe(df_obs))



footer="""<style>
    a:link , a:visited{
    color: blue;
    background-color: transparent;
    text-decoration: underline;
    }

    a:hover,  a:active {
    color: red;
    background-color: transparent;
    text-decoration: underline;
    }

    .footer {
    position: fixed;
    left: 0;
    bottom: 0;
    width: 100%;
    background-color: transparent;
    color: black;
    text-align: center;
    }
</style>
<div class="footer">
    <p>Developed by <a style='display: block; text-align: center;' href="https://www.linkedin.com/in/nsi%C3%A9ni-kouadio-eli%C3%A9zer-amany-613681185" target="_blank">Nsiéni Amany Kouadio</a></p>
</div>
"""
st.markdown(footer,unsafe_allow_html=True)

In [3]:
df.columns

Index(['debut', 'fin', 'text', 'titre', 'nom_CE', 'date_reporting', 'NbZD',
       'NumZD', 'UE_agent1', 'UE_agent2', 'UE_agent3', 'UE_total', 'note1',
       'UE F', 'UE I', 'partiels', 'refus', 'Nb_rdv', 'Nb_rdv_h',
       'Nb_rdv_attente', 'difficultes', 'observations', '_id', '_uuid',
       '_submission_time', '_validation_status', '_notes', '_status',
       '_submitted_by', '__version__', '_tags', '_index', 'Chef d'equipe',
       'ZD_affectees', 'date_2'],
      dtype='object')

In [5]:
df.Nb_rdv_attente.sum()

81.0

In [1]:
url = "https://kf.kobotoolbox.org/api/v2/assets/aJonRqpCqRSsja5S6H4dHb/export-settings/esZzBJnLtQCvVYVz7uhahgk/data.csv"

username = "amanyankey2"
password = "L@ScXi7TFhKWrN5"

# Effectuer une requête HTTP avec authentification basique
response = requests.get(url, auth=(username, password))

# Vérifier que la requête a réussi
if response.status_code == 200:
    df = function.get_data_from_forms(StringIO(response.text))
else:
    print("Aucune data")

NameError: name 'requests' is not defined